<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/FineTune_Llama2/Forecast_DaysToSettle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU \
    datasets transformers bitsandbytes accelerate peft trl

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
import datasets

dataset = datasets.load_dataset("mlabonne/guanaco-llama2-1k", split='train')
print(dataset[500])

{'text': '<s>[INST] What is the link between race and intelligence? [/INST] There is no scientific evidence to support a link between race and intelligence. Intelligence is a complex trait that is influenced by many factors, including genetics, environment, education, and culture. Race, on the other hand, is a social construct that is based on physical characteristics such as skin color, hair texture, and facial features. While there may be genetic differences between individuals of different races, these differences do not correspond to any meaningful differences in intelligence.\n\nIt is important to recognize that any attempts to link race and intelligence can have harmful and divisive effects on society. The notion that certain races are inherently more or less intelligent than others has been used throughout history to justify discrimination and oppression, and it has no scientific basis. It is essential to base our understanding of intelligence on rigorous scientific research, fr

In [13]:
import torch
import transformers
import peft
import trl

pretrained_model = "meta-llama/Llama-2-7b-chat-hf"

quantization_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=getattr(torch, 'float16'),
    bnb_4bit_use_double_quant=False
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model, quantization_config=quantization_config,
    device_map={"":0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = transformers.AutoTokenizer.from_pretrained(pretrained_model,
    use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = 'right'

training_args = transformers.TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type='constant',
    report_to='tensorboard'
)

peft_config = peft.LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64,
    bias='none', task_type='CAUSAL_LM'
)

trainer = trl.SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

trainer.train()

finetuned_model = "Llama2-7b-forecast_DaysToSettle"
trainer.model.save_pretrained(finetuned_model)
trainer.tokenizer.save_pretrained(finetuned_model)

ValueError: ignored

In [ ]:
import pandas as pd

